# Merging on Index

In some cases, the merge key or keys in a DataFrame will be found in its index. In this case, you can pass left_index=True or right_index=True (or both) to indicate that the index should be used as the merge key:

In [1]:
from pandas import DataFrame
import pandas as pd
import numpy as np

In [2]:
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                'value': range(6)})

In [3]:
right1 = DataFrame({'group_val': [3.5, 7]},
                    index=['a', 'b'])

In [4]:
left1, right1

(  key  value
 0   a      0
 1   b      1
 2   a      2
 3   a      3
 4   b      4
 5   c      5,
    group_val
 a        3.5
 b        7.0)

In [5]:
pd.merge(left1, right1, left_on = 'key', right_index = True, how = 'outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


With hierarchically-indexed data, things are a bit more complicated:

In [6]:
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                    'key2': [2000, 2001, 2002, 2001, 2002],
                    'data': np.arange(5.)})

In [7]:
righth = DataFrame(np.arange(12).reshape((6, 2)),
                    index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                    [2001, 2000, 2000, 2000, 2001, 2002]],
                    columns=['event1', 'event2'])

In [8]:
lefth, righth

(     key1  key2  data
 0    Ohio  2000   0.0
 1    Ohio  2001   1.0
 2    Ohio  2002   2.0
 3  Nevada  2001   3.0
 4  Nevada  2002   4.0,
              event1  event2
 Nevada 2001       0       1
        2000       2       3
 Ohio   2000       4       5
        2000       6       7
        2001       8       9
        2002      10      11)

In this case, you have to indicate multiple columns to merge on as a list (pay attention to the handling of duplicate index values):

In [9]:
pd.merge(lefth, righth, left_on = ['key1', 'key2'], right_index= True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [10]:
pd.merge(lefth, righth, left_on = ['key1', 'key2'], right_index= True, how = 'outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [23]:
pd.merge(lefth, righth, left_on = ['key1', 'key2'],
        right_index= True, how = 'outer',sort= False)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


Using the indexes of both sides of the merge is also not an issue:

In [24]:
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
                columns=['Ohio', 'Nevada'])

In [25]:
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                    index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])

In [26]:
right2, left2

(   Missouri  Alabama
 b       7.0      8.0
 c       9.0     10.0
 d      11.0     12.0
 e      13.0     14.0,
    Ohio  Nevada
 a   1.0     2.0
 c   3.0     4.0
 e   5.0     6.0)

In [36]:
pd.merge(left2, right2, how = 'outer', left_index= True, right_index= True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DataFrame has a more convenient join instance for merging by index. It can also be used to combine together many DataFrame objects having the same or similar indexes but non-overlapping columns. In the prior example, we could have written:

In [44]:
left2.join(right2, how = 'outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In part for legacy reasons (much earlier versions of pandas), DataFrame’s join method performs a left join on the join keys. It also supports joining the index of the passed DataFrame on one of the columns of the calling DataFrame:

In [45]:
left1.join(right1, on = 'key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


Lastly, for simple index-on-index merges, you can pass a list of DataFrames to join as an alternative to using the more general concat function described below:

In [46]:
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                    index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon'])

In [51]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [57]:
left2.join([right2, another], how= 'outer', sort=True)

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0
